In [151]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 19 13:29:30 2022

@author: xdata
"""
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import lightgbm as lgb
import math
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
import warnings
import pdb
warnings.filterwarnings('ignore')
import random
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
# from torch.utils.data import TensorDataset,SequentialSampler,RandomSampler,DataLoader
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [152]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 禁止hash随机化
    torch.manual_seed(seed)


set_seed(2022)
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print('训练集的大小为： ', len(train))
print('测试集的大小为： ', len(test))


train = train.dropna()
train.reset_index(drop=True,inplace=True)

label = 'Main_steam_flow_rate'

# =============================================================================
# ['时间', '主蒸汽流量', 'CO含量', 'HCL含量', 'NOx含量', 'SO2含量', '一次风调门', '一次风量',
#        '主蒸汽流量设定值', '二次风调门', '二次风量', '引风机转速', '推料器启停', '推料器手动指令',
#        '推料器自动投退信号', '推料器自动指令', '氧量设定值', '汽包水位', '炉排启停', '炉排实际运行指令',
#        '炉排手动指令', '炉排自动投退信号', '给水流量']
# =============================================================================


训练集的大小为：  257398
测试集的大小为：  1800


In [153]:
train.columns = ['Time', 'Main_steam_flow_rate', 'CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
       'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_start_stop', 'Pusher_manual_command',
       'Pusher_automatic_throwback_signal', 'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_start_stop', 'Grate_actual_operation_command',
       'Grate_manual_command', 'Grate_automatic_throw-out_signal', 'Feedwater_flow']

test.columns = ['Time', 'CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
       'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_start_stop', 'Pusher_manual_command',
       'Pusher_automatic_throwback_signal', 'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_start_stop', 'Grate_actual_operation_command',
       'Grate_manual_command', 'Grate_automatic_throw-out_signal', 'Feedwater_flow']


In [154]:
len(train.columns), len(test.columns)

(23, 22)

In [155]:
df = pd.concat([train, test])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259182 entries, 0 to 1799
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Time                               259182 non-null  object 
 1   Main_steam_flow_rate               257382 non-null  float64
 2   CO_content                         259182 non-null  float64
 3   HCL_content                        259182 non-null  float64
 4   NOx_content                        259182 non-null  float64
 5   SO2_content                        259182 non-null  float64
 6   Primary_air_regulator              259182 non-null  float64
 7   Primary_air_volume                 259182 non-null  float64
 8   Main_steam_flow_rate_setting       259182 non-null  float64
 9   Secondary_air_regulator            259182 non-null  float64
 10  Secondary_air_volume               259182 non-null  float64
 11  Inducer_fan_speed                  259182

In [156]:
df.head(5)


,Time,Main_steam_flow_rate,CO_content,HCL_content,NOx_content,SO2_content,Primary_air_regulator,Primary_air_volume,Main_steam_flow_rate_setting,Secondary_air_regulator,...,Pusher_manual_command,Pusher_automatic_throwback_signal,Pusher_automatic_command,Oxygen_setpoint,Ladle_water_level,Grate_start_stop,Grate_actual_operation_command,Grate_manual_command,Grate_automatic_throw-out_signal,Feedwater_flow
0,2021-12-20 00:00:06,54.4935,1.8403,5.9213,57.0556,2.8819,75.2477,72565.4141,60.0,21.1690,...,54.0946,True,54.0971,5.5,1.8370,True,44.0921,44.0946,True,74.3151
1,2021-12-20 00:00:07,54.4664,1.8681,5.9074,57.1111,2.8542,75.2384,71991.8828,60.0,21.3009,...,54.0435,True,54.0387,5.5,1.9551,True,44.1057,44.0435,True,74.3242
2,2021-12-20 00:00:08,54.5466,1.8264,5.8796,56.9444,2.8472,75.3657,71781.4844,60.0,21.3495,...,54.0453,True,54.0469,5.5,2.0904,True,44.0436,44.0453,True,74.2533
3,2021-12-20 00:00:09,54.5126,1.8403,5.8935,56.7593,2.9236,75.3426,72240.4453,60.0,21.4907,...,54.0535,True,54.0552,5.5,2.2391,True,44.0527,44.0535,True,74.2104
4,2021-12-20 00:00:10,54.3731,1.9028,5.9074,56.9259,2.8750,75.2940,72842.7813,60.0,21.7176,...,54.0618,True,54.0634,5.5,2.4038,True,44.0601,44.0618,True,74.1579


In [157]:
print('df.shape: ', df.shape)

df.drop(columns=['Time'], inplace=True)

df.shape:  (259182, 23)


In [158]:

df['gas'] = df.CO_content + df.HCL_content + df.SO2_content + df.NOx_content
for f in ['Feedwater_flow', 'Oxygen_setpoint', 'Primary_air_volume', 'Main_steam_flow_rate_setting', 'Ladle_water_level']:
    shift_f = []
    shift_d = []
    for i in range(200):
        colname = f+'_shift_{}'.format(i+1)
        df[colname] = df[f].shift(i+1)
        shift_f.append(colname)

        if i>10:
            shift_d.append(colname)
        colname = f+'_diff_{}'.format(i+1)
        df[colname] = df[f].diff(i+1)
# =============================================================================
#         if (i+1)%50==0:
#             data[f+'_fore_{}_steps_mean'.format(i)] = data[shift_f].mean(1)
#             data[f+'_fore_{}_steps_max'.format(i)] = data[shift_f].max(1)
#             data[f+'_fore_{}_steps_min'.format(i)] = data[shift_f].min(1)
#             data[f+'_fore_{}_steps_std'.format(i)] = data[shift_f].std(1)
# =============================================================================

    # data[f+'_diff'] = data[f].diff(1)
    df[f+'_fore_7_steps_mean'] = df[shift_f].mean(1)
    df[f+'_fore_7_steps_max'] = df[shift_f].max(1)
    df[f+'_fore_7_steps_min'] = df[shift_f].min(1)
    df[f+'_fore_7_steps_std'] = df[shift_f].std(1)
    df.drop(shift_d, axis=1, inplace=True)

df['Grate_start_stop'] = df['Grate_start_stop'].map(int)
df['Pusher_start_stop'] = df['Pusher_start_stop'].map(int)
df['Pusher_automatic_throwback_signal'] = df['Pusher_automatic_throwback_signal'].map(int)
df['Grate_automatic_throw-out_signal'] = df['Grate_automatic_throw-out_signal'].map(int)

In [189]:
for column in list(df.columns[df.isnull().sum() > 0]):

    mean_val = df[column].mean()

    df[column].fillna(0, inplace=True)

In [190]:
df_test = df.tail(1800)
df_train = df.iloc[:-1800, :]
df_train.reset_index(drop=True,inplace=True)

df_valid = df_train.iloc[-1800:, :]
df_train = df_train.iloc[:-1800, :]

df_valid.reset_index(drop=True, inplace=True)

In [208]:
df_train

,Main_steam_flow_rate,CO_content,HCL_content,NOx_content,SO2_content,Primary_air_regulator,Primary_air_volume,Main_steam_flow_rate_setting,Secondary_air_regulator,Secondary_air_volume,...,Ladle_water_level_diff_195,Ladle_water_level_diff_196,Ladle_water_level_diff_197,Ladle_water_level_diff_198,Ladle_water_level_diff_199,Ladle_water_level_diff_200,Ladle_water_level_fore_7_steps_mean,Ladle_water_level_fore_7_steps_max,Ladle_water_level_fore_7_steps_min,Ladle_water_level_fore_7_steps_std
0,54.4935,1.8403,5.9213,57.0556,2.8819,75.2477,72565.4141,60.0,21.1690,4898.9800,...,0.006806,0.006806,0.006806,0.006806,0.006806,0.006807,4.999725,8.832276,0.997382,2.302957
1,54.4664,1.8681,5.9074,57.1111,2.8542,75.2384,71991.8828,60.0,21.3009,4898.9800,...,0.006806,0.006806,0.006806,0.006806,0.006806,0.006807,1.837000,1.837000,1.837000,2.302957
2,54.5466,1.8264,5.8796,56.9444,2.8472,75.3657,71781.4844,60.0,21.3495,4898.9800,...,0.006806,0.006806,0.006806,0.006806,0.006806,0.006807,1.896050,1.955100,1.837000,0.083509
3,54.5126,1.8403,5.8935,56.7593,2.9236,75.3426,72240.4453,60.0,21.4907,5123.4751,...,0.006806,0.006806,0.006806,0.006806,0.006806,0.006807,1.960833,2.090400,1.837000,0.126797
4,54.3731,1.9028,5.9074,56.9259,2.8750,75.2940,72842.7813,60.0,21.7176,5123.4751,...,0.006806,0.006806,0.006806,0.006806,0.006806,0.006807,2.030400,2.239100,1.837000,0.173426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255577,68.3063,300.0692,1.7870,30.4815,202.0139,30.3401,32727.2578,60.0,89.4491,35028.5625,...,24.957900,25.175700,25.528300,25.788700,25.870100,25.961200,12.305163,26.686700,-2.627900,10.013213
255578,68.4041,300.0969,1.8287,30.3704,201.9236,30.3169,32641.5645,60.0,89.4097,34996.4297,...,24.824300,24.921500,25.139300,25.491900,25.752300,25.833700,12.434969,26.686700,-2.536800,9.986895
255579,68.2642,300.1178,1.8148,30.3519,201.9028,30.3077,32693.0098,60.0,89.4144,34781.4570,...,24.839500,24.814200,24.911400,25.129200,25.481800,25.742200,12.564137,26.686700,-2.455400,9.959306
255580,68.2996,300.0693,1.8056,30.5926,201.8333,30.3470,32812.7305,60.0,89.4259,34565.1523,...,24.670100,24.789300,24.764000,24.861200,25.079000,25.431600,12.692848,26.686700,-2.195000,9.930523


In [191]:
print('df_train.shape: ', df_train.shape)
print('df_valid.shape: ', df_valid.shape)
print('df_test.shape: ', df_test.shape)

df_train.shape:  (255582, 1098)
df_valid.shape:  (1800, 1098)
df_test.shape:  (1800, 1098)


In [192]:
scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_train)
df_for_validing_scaled = scaler.transform(df_valid)
# df_for_testing_scaled=scaler.transform(df_test)

In [193]:
df_for_training_scaled.shape

(255582, 1098)

In [194]:
trainX, trainY = df_for_training_scaled[:, 1:], df_for_training_scaled[:, 0]
validX, validY = df_for_validing_scaled[:, 1:], df_for_validing_scaled[:, 0]

In [195]:
print('构建时间序列特征')
print('trainX.shape, trainY.shape: ', trainX.shape, trainY.shape)
print('validX.shape, validY.shape: ', validX.shape, validY.shape)
# print('testX.shape, testY.shape: ', testX.shape, testY.shape)

构建时间序列特征
trainX.shape, trainY.shape:  (255582, 1097) (255582,)
validX.shape, validY.shape:  (1800, 1097) (1800,)


In [196]:
class My_Dataset(Dataset):
    def __init__(self, data, labels, test_mode=False):
        self.data = data
        self.labels = labels
        self.test_mode = test_mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.test_mode:
            sample = dict(input=torch.FloatTensor(self.data[idx]))
        else:
            sample = dict(input=torch.FloatTensor(self.data[idx]),
                          label=torch.FloatTensor(np.array(self.labels[idx])))
        return sample


In [197]:
def create_data_loader(data=None, labels=None, batch_size=32, test_mode=False):
    ds=My_Dataset(
        data = data,
        labels = labels,
        test_mode=test_mode)
    return DataLoader(ds, batch_size=batch_size, shuffle=False)
    # if test_mode:
    #     return DataLoader(ds, batch_size=batch_size, shuffle=False)
    # else:
    #     return DataLoader(ds, batch_size=batch_size, shuffle=True)

In [198]:
train_data_loader = create_data_loader(data=trainX,
                                       labels=trainY)

val_data_loader = create_data_loader(data=validX,
                                       labels=validY)

In [199]:
len(train_data_loader)

7987

In [200]:
data = next(iter(train_data_loader))
data.keys()
print(data['input'].shape)
print(data['label'].shape)

torch.Size([32, 1097])
torch.Size([32])


In [201]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('use device: ', device)

use device:  cuda:0


In [202]:
class Time_Model(nn.Module):

    def __init__(self):
        super().__init__()

        # self.hidden_size = 256
        # self.num_layers = 1
        # self.bidirectional = True
        # self.lstm = nn.LSTM(input_size=22,
        #                      hidden_size=self.hidden_size,
        #                      num_layers=self.num_layers,
        #                      bidirectional=self.bidirectional,
        #                      batch_first=True)
        # self.fc = nn.Linear(self.hidden_size*2, 1)
        feature_dim = 1097
        self.mlp = nn.Sequential(
            nn.Linear(feature_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.ReLU(),
        )

    def forward(self, x):
        # device = x.device
        # batch_size, seq_len, emb_dim = x.shape
        # # 初始化：双向就乘2
        # h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size).to(device)
        # c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size).to(device)
        # x, (_, _) = self.lstm(x, (h0, c0))
        # x = x[:,-1,:] # 取最后一个时间步
        # x = self.fc(x)
        x = self.mlp(x)
        return x

In [203]:
model = Time_Model()
model = model.to(device)

In [204]:
# MSE开方就是RMSE了，所以损失可以直接用来作为评估指标哦
loss_fn = torch.nn.MSELoss(reduction='mean').to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 32

In [205]:
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []

    pred_list = []
    target_list = []

    for inputs in tqdm(data_loader):
        targets = inputs["label"].to(device)
        x = inputs['input'].to(device)
        outputs = model(x)
        preds = outputs


        pred_list.extend(preds.cpu().detach().numpy().tolist())
        target_list.extend(targets.cpu().detach().numpy().tolist())


        loss = loss_fn(outputs, targets)
        # MSE开方就是RMSE了
        losses.append(np.sqrt(loss.item()))
        loss.backward()



        # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()
        # scheduler.step()

    mean_loss = np.mean(losses)


    return mean_loss

In [206]:
def eval_model(model, data_loader, loss_fn, device):
    model = model.eval() # 验证预测模式
    losses = []
    pred_list = []
    target_list = []

    with torch.no_grad():
        for inputs in tqdm(data_loader):
            targets = inputs["label"].to(device)
            x = inputs['input'].to(device)
            outputs = model(x)
            preds = outputs


            pred_list.extend(preds.cpu().detach().numpy().tolist())
            target_list.extend(targets.cpu().detach().numpy().tolist())


            loss = loss_fn(outputs, targets)


            # MSE开方就是RMSE了
            losses.append(np.sqrt(loss.item()))



    mean_loss = np.mean(losses)

    return mean_loss

In [207]:

from collections import defaultdict
history = defaultdict(list) # 记录10轮loss和acc
best_rmse = float('inf')


# -------------------控制早停--------------
early_stop_epochs = 5
no_improve_epochs = 0


for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_rmse = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
    )

    print(f'train_rmse : {train_rmse } \n ')

    val_rmse = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device
    )

    print(f'val_rmse : {val_rmse } \n ')
    print()

    history['train_rmse '].append(train_rmse)
    history['val_rmse '].append(val_rmse)


    if val_rmse  < best_rmse :
        print('best model saved!!!!!!!!!!!!!')
        torch.save(model.state_dict(), f'./save model/best_model.pt')
        best_rmse  = val_rmse

        no_improve_epochs = 0

    else:
        no_improve_epochs += 1



    if no_improve_epochs == early_stop_epochs:
        print('no improve score !!! stop train !!!')
        break

Epoch 1/32
----------
train_rmse : 0.4220077037134499 
 
val_rmse : 0.4773867655171617 
 

best model saved!!!!!!!!!!!!!
Epoch 2/32
----------
train_rmse : 0.4218875788798358 
 
val_rmse : 0.4773867655171617 
 

Epoch 3/32
----------


 44%|████▎     | 3493/7987 [00:06<00:07, 581.85it/s]


KeyboardInterrupt: 

In [ ]:
df_test

In [ ]:

df_test.info()
df_test['Main_steam_flow_rate'] = 0
df_test.head(5)

In [ ]:

new_scaled_array=scaler.transform(df_test)
new_scaled_df=pd.DataFrame(new_scaled_array)
print('new_scaled_array.shape: ', new_scaled_array.shape)
new_scaled_df.iloc[:,0]=np.nan

In [ ]:
testX = new_scaled_df.values[:, 1:]
testX.shape

In [ ]:
test_data_loader = create_data_loader(data=testX,
                                      labels=None,
                                      test_mode=True)



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('use device: ', device)
model = Time_Model()


path = './save model/best_model.pt'
model.load_state_dict(torch.load(path))
model = model.to(device)


model.eval()

In [ ]:
pred_list = []
with torch.no_grad():
    for inputs in tqdm(test_data_loader):
        x = inputs['input'].to(device)
        preds = model(x)
        pred_list.extend(preds.cpu().detach().numpy().tolist())

In [ ]:
len(pred_list)

In [ ]:
new_array=np.array(pred_list)
new_array=new_array.reshape(-1,1)

In [ ]:
prediction_copies_array = np.repeat(new_array,1098, axis=-1)
y_pred_future_days = scaler.inverse_transform(np.reshape(prediction_copies_array,(len(new_array),1098)))[:,0]
print(y_pred_future_days)

In [ ]:
test = pd.read_csv('data/test.csv')

In [ ]:

sub = pd.DataFrame({'ID': list(range(1,1801)),
                    'Time': test['时间'],
                    'Steam_flow': y_pred_future_days})
sub.to_csv('result.csv',index=False)

In [ ]:

sub = pd.read_csv('./demo2.csv')

In [ ]:
import matplotlib.pyplot as plt
train = pd.read_csv('data/train.csv')
t1 = train['主蒸汽流量'].tolist()
t2 = sub['Steam_flow'].tolist()
plt.figure(dpi=200)
plt.plot(list(range(len(t1))), t1)
plt.plot(list(range(len(t1), len(t1)+len(t2))), t2)
plt.savefig('./pred.jpg')

In [50]:
list(range(t1))

TypeError: 'list' object cannot be interpreted as an integer